## Multi-Agentic Framework

*copied code from __[lanceDB](https://blog.lancedb.com/multi-document-agentic-rag-a-walkthrough/)__ multi-agentic for proof of concept*

In [ ]:
import os
import tqdm
import json
import time
from typing import List, Optional, Dict, Any
from tqdm import tqdm
import logging
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    Settings,
    Document,
)
from datetime import datetime, timedelta
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import FunctionTool, ToolOutput
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

*I copied the imports so all of them are probably unecessary*

In [ ]:
tablenameVectorStore = LanceDBVectorStore(
    uri='./lancedb',
    table_name='',
    mode="overwrite",
)

#followed by any other task specific Vector Storage space 

# Index Json

In [ ]:
def loadAndIndexDocument(file_path: str, vector_store: LanceDBVectorStore) -> VectorStoreIndex:
    '''load document'''

    with open(file_path,'r') as f:
        data = json.load(f)
        document = Document(text = json.dumps(data))
        
    parser = SentenceSplitter(chunk_size = 1024, chunk_overlap = 200)
    nodes = parser.get_nodes_from_documents([document])
    storageContext = StorageContext.from_defaults(vector_store=vector_store)
    return VectorStoreIndex(nodes, storageContext=storageContext)

nameof_index = loadAndIndexDocument(
    "problems.json", problems_vector_store
)

# Retriver

In [ ]:
def createRetriver(index: VectorStoreIndex)-> VectorIndexRetriever:
    return index.as_retriever()

Create retrievers -
nameof_retriever = create_retriever(document_index)